In [8]:
import numpy as np
import qiskit
from numpy import linalg as LA
import matplotlib.pyplot as plt
import scipy
import qib
import h5py
import sys
import rqcopt as oc
from scipy.sparse.linalg import expm_multiply


Lx, Ly = (6, 6)
L = Lx*Ly
t = .125
latt = qib.lattice.IntegerLattice((Lx, Ly), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
J, h, g = (1, 0, 1)


X = np.array([[0, 1], [1, 0]])
Z = np.array([[1, 0], [0, -1]])
Y = np.array([[0, -1j], [1j, 0]])
I2 = np.array([[1, 0], [0, 1]])

perms_v, perms_h = (
    [[0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35],
    [1, 2, 3, 4, 5, 0, 7, 8, 9, 10, 11, 6, 13, 14, 15, 16, 17, 12, 19, 20, 21, 22, 23, 18, 25, 26, 27, 28, 29, 24, 31, 32, 33, 34, 35, 30]],
    [[0, 6, 12, 18, 24, 30, 1, 7, 13, 19, 25, 31, 2, 8, 14, 20, 26, 32, 3, 9, 15, 21, 27, 33, 4, 10, 16, 22, 28, 34, 5, 11, 17, 23, 29, 35], 
    [6, 12, 18, 24, 30, 0, 7, 13, 19, 25, 31, 1, 8, 14, 20, 26, 32, 2, 9, 15, 21, 27, 33, 3, 10, 16, 22, 28, 34, 4, 11, 17, 23, 29, 35, 5]]
)

In [9]:
"""
    Compressed-Controlled Time Evolution Operator that we optimized previously.
"""
import h5py


Xlists_opts = {}
Vlists = {}
qc_cUs = {}
ts = [.125]
for t in [.125]:
    Vlist = []

    with h5py.File(f"./results/tfim2d_ccU_SPARSE_101_Lx4Ly4_t{t}_layers6_niter8_rS1_2hloc.hdf5", "r") as f:
        Vlist =  f["Vlist"][:]
    perms_extended = [[perms_v[0]]] + [perms_v]*1 + [[perms_v[0]], [perms_h[0]]] +\
                        [perms_h]*1 + [[perms_h[0]]]
    perms_ext_reduced = [perms_v]*1  + [perms_h]*1
    control_layers = [0, 2, 3, 5]           # 4 control layers
    
    perms_qc = [[0, 1], [0, 2]]
    Xlists_opt = {}
    for i in control_layers:
        with h5py.File(f"./results/tfim2d_ccU_SPARSE_{J}{h}{g}_Lx4Ly4_t{t}_layers{len(Vlist)}_niter15_rS1_DECOMPOSE_n{len(perms_qc)}_layer{i}.hdf5", "r") as file:
            Xlists_opt[i] = file[f"Xlist_{i}"][:]
    
    Xlists_opts[t] = Xlists_opt
    Vlists[t] = Vlist


In [10]:
import random
from pytket import Circuit, Qubit
from pytket.pauli import Pauli
from pytket.circuit import Circuit, Unitary1qBox, Unitary2qBox


def construct_ccU(L, Vs, Xlists_opt, perms, perms_qc, control_layers):
    qc = Circuit(L+1)
    qc.X(0)
    for i, V in enumerate(Vs):
        layer = i
        if i in control_layers:
            for perm in perms[layer]:
                for j in range(L//2):
                    mapp = {0: 0, 1: perm[2*j]+1, 2:perm[2*j+1]+1}
                    for g, G in enumerate(Xlists_opt[i]):
                        qc.add_unitary2qbox(Unitary2qBox(G), mapp[perms_qc[g][0]], mapp[perms_qc[g][1]])
        else:
            for perm in perms[layer]:
                for j in range(L//2):
                    qc.add_unitary2qbox(Unitary2qBox(V), perm[2*j]+1, perm[2*j+1]+1)
                    
    qc.X(0)
    return qc

qc_cU = construct_ccU(L, Vlists[0.125], Xlists_opts[0.125], perms_extended, perms_qc, control_layers)

In [11]:
"""
    Adiabatic Evolution Implementation.
"""

def trotter(Lx, Ly, tau, L, J_i, h_i, g_i, J_f, h_f, g_f, lamb):
    L = Lx * Ly
    assert lamb <= 1 and lamb >= 0
    J = lamb*J_f + (1-lamb)*J_i
    g = lamb*g_f + (1-lamb)*g_i
    h = lamb*h_f + (1-lamb)*h_i

    qc = Circuit(L)
    hloc = construct_ising_local_term_(J, g, 2)
    perm_set = perms_v + perms_h
    perms = perm_set
    
    method_start = oc.SplittingMethod.suzuki(len(perm_set), 1)
    indices = method_start.indices
    coeffs = method_start.coeffs

    Vlist_start = []
    perms = []
    for i, c in zip(indices, coeffs):
        Vlist_start.append(scipy.linalg.expm(-1j*c*tau*hloc))
        perms.append(perm_set[i])

    for layer, V in enumerate(Vlist_start):     
        for j in range(len(perms[layer])//2):
            qc.add_unitary2qbox(Unitary2qBox(V), perms[layer][2*j], perms[layer][2*j+1])
    return qc


def construct_ising_local_term_(J, g, ndim, h=0):
    X = np.array([[0.,  1.], [1.,  0.]])
    Z = np.array([[1.,  0.], [0., -1.]])
    I = np.identity(2)
    return J*np.kron(Z, Z) + g*(0.5/ndim)*(np.kron(X, I) + np.kron(I, X)) + h*(0.5/ndim)*(np.kron(Z, I) + np.kron(I, Z))


def run_adiabatic(Lx, Ly, g, T, S, init_circ, h_i=0, h_f=0):
    L = Lx*Ly
    tau = 1/S
    t_s = np.linspace(0, T, S*T)
    sch = lambda t, T: np.sin(np.pi*t/(2*T))**2
    
    qc = init_circ.copy()
    for s in range(S*T):
        qc.append(trotter(Lx, Ly, tau, L, J, h_i, 0, J, h_f, g, sch(t_s[s], T)))
    #c = backend.get_compiled_circuit(qc)
    #handle = backend.process_circuit(c)
    #final = backend.get_result(handle).get_state()
    #print("AQC: ", [np.linalg.norm(np.vdot(final, eigenvectors_sort[:, i]))**2 for i in range(10)])
    return qc

In [19]:
qc_C = Circuit(L)
for y in range(Ly):
    for x in range(Lx):
        i = Ly * y + x  # qubit index in row-major order
        if (x + y) % 2 == 1:
            qc_C.X(i)  # Flip qubit to |1⟩ (spin down)
qc_A = run_adiabatic(Lx, Ly, g, 2, 2, qc_C)
qc_A.rename_units({Qubit(i): Qubit(i + 1) for i in range(L)})

True

In [38]:
!qnx login

🌐 Browser log in initiated.
╭──────────────────────────────────────────────────────────────────────────────╮
│                                                                              │
│         Confirm that the browser shows the following code and click 'allow   │
│ device':                                                                     │
│                                                                              │
│                                      Nbsfzv                                  │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯
Browser didn't open automatically? Use this link: https://nexus.quantinuum.com/auth/device/browser?otp=NbsfzvWVQU_1o9H546ff1W0Q7-OizxPpcTUlhFSnM-XIh4zb8adUK0pWN2nyBcKlnKtvyAuJpjAxXBlvD3EqVQ
✅ Successfully logged in as ekaracan@student.ethz.ch using the browser.



In [13]:
import qnexus as qnx

my_project_ref = qnx.projects.get_or_create(name="Compile Tester")

In [18]:
from pytket.circuit import Unitary1qBox

for t in [0.125]:
    qc_cU = construct_ccU(L, Vlists[t], Xlists_opts[t], perms_extended, perms_qc, control_layers)
    qc_QPE_real = Circuit(L+1, 1)
    qc_QPE_real.append(qc_A)
    qc_QPE_real.H(0)
    qc_QPE_real.append(qc_cU)
    qc_QPE_real.H(0)
    qc_QPE_real.Measure(0, 0)
    
    qc_QPE_imag = Circuit(L+1, 1)
    qc_QPE_imag.append(qc_A)
    qc_QPE_imag.H(0)
    qc_QPE_imag.append(qc_cU)
    qc_QPE_imag.add_unitary1qbox(Unitary1qBox(np.array([[1, 0],
                                                        [0, -1j]])), 0)
    qc_QPE_imag.H(0)
    qc_QPE_imag.Measure(0, 0)

    # Upload Circuits
    circ_refs = []
    circ_refs.append(qnx.circuits.upload(
        name=f"6x6, QPE Real, t={t}",
        circuit=qc_QPE_real,
        project=my_project_ref,
    ))
    
    circ_refs.append(qnx.circuits.upload(
        name=f"6x6, QPE Imag, t={t}",
        circuit=qc_QPE_imag,
        project=my_project_ref,
    ))

    # Compile Circuits
    compiled_circuits = qnx.compile(
        circuits=circ_refs,
        name=f"6x6, QPE 6-layers ccU compressed T, S = (0, 2), t={t}  2-OPTIMIZED_v1",
        optimisation_level = 2,
        backend_config=qnx.QuantinuumConfig(device_name="H1-Emulator"),
        project=my_project_ref,
    )

    
    execute_job_ref = qnx.start_execute_job(
        circuits=compiled_circuits,
        name=f"6x6, QPE 6-layers ccU compressed T, S = (0, 2), t={t}, Noisy 2-OPTIMIZED",
        n_shots=[4000]*2,
        backend_config=qnx.QuantinuumConfig(device_name="H1-Emulator"),
        project=my_project_ref,
    )    
    execute_job_ref.df()

    
    """
    execute_job_ref = qnx.start_execute_job(
        circuits=compiled_circuits,
        name=f"6x6, QPE 10-layers ccU compressed T, S = (1, 1), t={t}, Noiseless 2--OPTIMIZED   ",
        n_shots=[4000]*2,
        backend_config=qnx.QuantinuumConfig(device_name="H1-1E"),
        project=my_project_ref,
    )
    
    execute_job_ref.df()"""

ResourceCreateFailed: Failed to create resource with status code: 403, message: {"message":"Reached monthly quota of simulation time for this org."}

In [ ]:
import pandas as pd

counts_real = {}
counts_imag = {}

for i, id in enumerate(range(2, 10)):
    table_csv = pd.read_csv(f'./Emulator/shot_results_t0.25_real.csv')

    counts_real[0.25]={}
    
    
    for index, row in table_csv.iterrows():
        str_key = row['Outcome']
        count =  int(row['Count'].replace(',', ''))  if type(row['Count'])==str else int(row['Count'])
        counts_real[0.25][int(str_key)] = count

In [ ]:
from pytket.extensions.qiskit import AerBackend

backend = AerBackend()

compiled_circ = backend.get_compiled_circuit(qc_QPE_real)
handle = backend.process_circuit(compiled_circ, n_shots=2000)
counts_real = backend.get_result(handle).get_counts()

compiled_circ = backend.get_compiled_circuit(qc_QPE_imag)
handle = backend.process_circuit(compiled_circ, n_shots=2000)
counts_imag = backend.get_result(handle).get_counts()

In [347]:

counts  = {
           0.2:  [{(0, ): 1106, (1, ): 894}, {(0, ): 1620, (1, ): 380}],
           0.21: [{(0, ): 505, (1, ): 1495}, {(0, ): 1436, (1, ): 564}],
    
           0.22: [{(0, ): 410, (1, ): 1590}, {(0, ): 814, (1, ): 1186}],
    
           0.23: [{(0, ): 876, (1, ): 1124}, {(0, ): 391, (1, ): 1609}],
           0.5 : [{(0, ): 1015, (1, ): 985}, {(0, ): 1552, (1, ): 448}]
          }

In [348]:
phases_est = {}
for t in [0.2, 0.21, 0.22, 0.23, 0.5]:
    counts_real, counts_imag = counts[t]
    phase_est_real = ((counts_real[(0, )] if (0, ) in counts_real else 0) - (counts_real[(1, )] if (1, ) in counts_real else 0)) /\
                        ((counts_real[(0, )] if (0, ) in counts_real else 0) + (counts_real[(1, )] if (1, ) in counts_real else 0))     
    phase_est_imag = ((counts_imag[(0, )] if (0, ) in counts_imag else 0) - (counts_imag[(1, )] if (1, ) in counts_imag else 0)) /\
                        ((counts_imag[(0, )] if (0, ) in counts_imag else 0) + (counts_imag[(1, )] if (1, ) in counts_imag else 0))
    phases_est[t] = phase_est_real + 1j*phase_est_imag
    print("Estimated Phase Amplitude: ", np.linalg.norm(phase_est_real + 1j*phase_est_imag))

Estimated Phase Amplitude:  0.6289960254246445
Estimated Phase Amplitude:  0.6596370213988902
Estimated Phase Amplitude:  0.6186242801571887
Estimated Phase Amplitude:  0.6214957763331944
Estimated Phase Amplitude:  0.55220376673833


In [349]:
phases_est

{0.2: (0.106+0.62j),
 0.21: (-0.495+0.436j),
 0.22: (-0.59-0.186j),
 0.23: (-0.124-0.609j),
 0.5: (0.015+0.552j)}

In [45]:
# For t=0.2, here is the exact phase:   (-0.15823157287931242+0.9874020302511732j)
# For t=0.21, here is the exact phase:  (-0.9275722326124037+0.37364388565376094j)
# For t=0.22, here is the exact phase:  (-0.7984811662380588-0.6020197896773075j)
# For t=0.23, here is the exact phase:  (0.10400607051011165-0.9945766623529056j)
# For t=0.25, here is the exact phase:  (0.8302036981146572+0.5574601507163961j)
# For t=0.5, here is the exact phase:  (0.3784763607264664+0.9256109573526287j)
# 

In [6]:
import random
from pytket import Circuit, Qubit
from pytket.pauli import Pauli

def random_state_prep_circuit(n_qubits: int, depth: int = 10) -> Circuit:
    circ = Circuit(n_qubits)

    for d in range(depth):
        # Layer of random single-qubit rotations
        for q in range(n_qubits):
            theta = random.uniform(0, 2 * 3.1415)
            phi = random.uniform(0, 2 * 3.1415)
            lamb = random.uniform(0, 2 * 3.1415)
            circ.Rz(phi, q)
            circ.Ry(theta, q)
            circ.Rz(lamb, q)

        # Layer of CX gates (entangling)
        for q in range(0, n_qubits - 1, 2):
            circ.CX(q, q + 1)

        # Optionally shuffle qubits between layers for more mixing
        random.shuffle(list(range(n_qubits)))

    return circ


def reverse_bits_statevector(statevec, num_qubits):
    n = len(statevec)
    output = np.zeros(n, dtype=complex)
    for i in range(n):
        reversed_i = int(f"{i:0{num_qubits}b}"[::-1], 2)
        output[reversed_i] = statevec[i]
    return output

In [6]:
import numpy as np
from scipy.linalg import eig, qr

def demultiplex(U, V):
    """Compute the type-A KAK decomposition of the block-diagonal matrix U⊕V that
    corresponds to demultiplexing."""
    delta = U @ V.conj().T
    # Compute eigenvalue decomposition
    D_squared, U_1 = eig(delta)
    # Compute the square root by extracting phases and halving them
    phi = np.angle(D_squared) / 2
    U_2 = np.diag(np.exp(1j * phi)) @ U_1.conj().T @ V
    # Return the rotation angles for A, instead of the diagonal matrix D
    return U_1, phi, U_2

In [7]:
from scipy.linalg import cossin, eig, qr
import pennylane as qml

def demultiplex(U, V):
    """Numerically stable demultiplexing decomposition: K = K1 · A · K2"""
    delta = U @ V.conj().T
    # Eigen decomposition of delta (not necessarily Hermitian)
    D_squared, U_1 = eig(delta)
    # Fix: Re-orthogonalize U_1 (ensure unitary)
    U_1, _ = qr(U_1)
    # Compute eigenphases, halve them
    phi = np.angle(D_squared) / 2
    # Construct U_2
    U_2 = np.diag(np.exp(1j * phi)) @ U_1.conj().T @ V
    # Fix: Project U_2 back to nearest unitary using SVD
    U, _, Vh = np.linalg.svd(U_2)
    U_2 = U @ Vh
    return U_1, phi, U_2

In [68]:
from qiskit import QuantumCircuit, transpile

Vlist = Vlists[0.25]
decomposed_Vs = []
counts_nc = 0
counts_c = 0
for i in range(15):
    if i not in control_layers:
        qc = qiskit.QuantumCircuit(2)
        qc.unitary(Vlist[i], [0, 1])
        transpiled_circuit = transpile(qc, basis_gates=['rzz', 'u3'])
        gate_counts = transpiled_circuit.count_ops()
        counts_nc += gate_counts['rzz']
        decomposed_Vs.append(transpiled_circuit)

        from qiskit_aer import AerSimulator
        from qiskit_aer.library import SaveUnitary
        # Add instruction to save unitary
        qc.save_unitary()
        simulator = AerSimulator(method='unitary')
        transpiled = transpile(qc, simulator)
        result = simulator.run(transpiled).result()
        unitary = result.data(0)['unitary']
        assert np.allclose(np.array(unitary.data), Vlist[i])
    
    else:
        ket_0 = np.array([[1],[0]])
        ket_1 = np.array([[0],[1]])
        rho_0_anc = ket_0 @ ket_0.T
        rho_1_anc = ket_1 @ ket_1.T
        K_1 = np.kron(rho_0_anc, np.eye(4)) + np.kron(rho_1_anc, Vlist[i])
        n = 3
        zero = np.zeros((4, 4))
        
        U_1, phi, U_2 = demultiplex(np.eye(4), Vlist[i])
        
        rz_ops = [qml.RZ(-2 * p, 0) for p in phi]
        demultiplex_A = qml.matrix(qml.Select(rz_ops, control=range(1, n)), wire_order=range(n))
        #demultiplex_K_1 = np.block([[U_1, zero], [zero, U_1]])
        #demultiplex_K_2 = np.block([[U_2, zero], [zero, U_2]])
        demultiplex_K_1 = np.kron(np.eye(2), U_1)
        demultiplex_K_2 = np.kron(np.eye(2), U_2)
        
        reconstructed_K_1 = demultiplex_K_1 @ demultiplex_A @ demultiplex_K_2
        assert np.allclose(reconstructed_K_1, K_1)
        
        qc = qiskit.QuantumCircuit(3)
        qc.unitary(U_2, [0, 1], label='U2')
        qc.unitary(demultiplex_A  , [0, 1, 2], label='A')
        qc.unitary(U_1, [0, 1], label='U1')
        if i==0:
            print(qc.draw())
        
        transpiled_circuit = transpile(qc, basis_gates=['rzz', 'u3'])
        gate_counts = transpiled_circuit.count_ops()
        counts_c += gate_counts['rzz']
        decomposed_Vs.append(transpiled_circuit)

        from qiskit_aer import AerSimulator
        from qiskit_aer.library import SaveUnitary
        # Add instruction to save unitary
        qc.save_unitary()
        simulator = AerSimulator(method='unitary')
        transpiled = transpile(qc, simulator)
        result = simulator.run(transpiled).result()
        unitary = result.data(0)['unitary']
        #print(gate_counts['rzz'])
        assert np.allclose(np.array(unitary.data), K_1)
        #decomposed_Vs.append((U_1, A, U_2))
        
counts_nc*16 + counts_c*8

     ┌─────┐┌────┐┌─────┐
q_0: ┤0    ├┤0   ├┤0    ├
     │  U2 ││    ││  U1 │
q_1: ┤1    ├┤1 A ├┤1    ├
     └─────┘│    │└─────┘
q_2: ───────┤2   ├───────
            └────┘       


1192

In [ ]:
from pytket.extensions.qiskit import qiskit_to_tk
from pytket.circuit.display import render_circuit_jupyter

render_circuit_jupyter(qiskit_to_tk(decomposed_Vs[6]))